# Lab 2 - Diversity

## Required dependencies

In [ ]:
%pip install jmetalpy

In [ ]:
import numpy as np 
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

In [ ]:
import logging

logger = logging.getLogger('jmetal.core.algorithm')
logger.setLevel(logging.INFO)

## Different types of plots

When working with evolutionary algorithms, it is often necessary to visualize the population of solutions in order to gain insights into the optimization process. There are various types of plots and libraries that can be used for this purpose, and in this context, we will explore some of the most useful ones.

Visualizing the population can help us to understand how the algorithm is exploring the search space, how the fitness values are distributed, and how the diversity of the population changes over time. This information can be used to guide the selection of appropriate algorithm parameters, such as mutation rates and selection strategies, and to diagnose potential issues with the optimization process.

Let's start by generating a random population:

In [2]:
np.random.seed(42)
POPULATION_SIZE = 50
NUMBER_OF_DIMENSIONS = 3
population = np.random.rand(POPULATION_SIZE, NUMBER_OF_DIMENSIONS)

Let's consider an example where we have a population of solutions that can be divided into three groups (A, B, and C). To assign each solution to a group randomly, we can use a `np.random.choice` function:

In [ ]:
groups = np.random.choice(["A", "B", "C"], POPULATION_SIZE)

In [ ]:
population[0], groups[0]

### Scatter plots (2D and 3D)
A scatter plot is a type of plot used to visualize the relationship between two (or three) variables.

In [ ]:
plt.scatter(population[:, 0], population[:, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.title("Population")
plt.show()

Scatter plot with different colors for each group:

In [ ]:
fig, ax = plt.subplots()
for group in np.unique(groups):
    group_population = population[groups == group]
    x = group_population[:, 0]
    y = group_population[:, 1]
    ax.scatter(x, y, label=group)
ax.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Population (different colors for each group)")
plt.show()

3D scatter plot:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(population[:, 0], population[:, 1], population[:, 2])
plt.show()

Both `matplotlib` and `plotly` are powerful Python libraries for creating visualizations. However, there are some cases where `plotly` may be more convenient than `matplotlib`.

One advantage of `plotly` is that it provides interactive visualizations that can be easily embedded in web applications or shared online. `plotly` also has a more modern and user-friendly interface for creating visualizations, with features such as zooming, panning, and hover-over tooltips.

Another advantage of `plotly` is that it has built-in support for creating 3D visualizations, which can be useful for visualizing high-dimensional data. `matplotlib` also has some support for 3D visualizations, but it can be more difficult to use and less flexible than `plotly`.

In [ ]:
fig = px.scatter(x=population[:, 0], y=population[:, 1]) 
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
fig = px.scatter_3d(x=population[:, 0], y=population[:, 1], z=population[:, 2]) 
fig.show()

### High-dimensional data
- In optimization problems, the goal is to find the best solution to a problem within a given set of constraints. Often, the solution space can be very large and complex, with many variables and constraints that need to be considered. In such cases, the problem can be formulated as a high-dimensional optimization problem, where the number of variables is very large.
- Visualizing highly dimensional data can be challenging, as it is difficult to represent more than three dimensions in a 2D or 3D plot. However, there are several techniques that can be used to visualize high-dimensional data.

In [ ]:
POPULATION_SIZE = 15
high_dimensional_population = np.random.rand(POPULATION_SIZE, 10)
high_dimensional_groups = np.random.choice(["A", "B", "C"], POPULATION_SIZE)

#### Parallel coordinates
`parallel_coordinates` is a visualization technique that is used to plot high-dimensional data in a 2D space.

Each variable is represented by a vertical axis, and each data point is represented by a line that connects the values of the variables. The lines are parallel to each other, hence the name "parallel coordinates". The values of each variable are normalized to fit within the same range, so that they can be plotted on the same scale.

In [ ]:
high_dimensional_df = pd.DataFrame(high_dimensional_population, columns=[f'x{i}' for i in range(1, 11)])
high_dimensional_df["group"] = high_dimensional_groups
pd.plotting.parallel_coordinates(high_dimensional_df, "group")
plt.show()

The `parallel_coordinates` plot can sometimes be difficult to interpret, especially when the number of variables is large. This is because the lines can become tangled and difficult to distinguish from each other, making it hard to see the patterns and relationships between the variables.

#### Selecting 2-3 dimensions

In [ ]:
fig = px.scatter_3d(x=high_dimensional_population[:, 0], y=high_dimensional_population[:, 1], z=high_dimensional_population[:, 2]) 
fig.show()

#### Radar (spider) chart
Spider chart are frequently used to compare different models: https://huggingface.co/HuggingFaceH4/zephyr-7b-beta.

In [ ]:
high_dimensional_df = pd.DataFrame(high_dimensional_population, columns=[f'x{i}' for i in range(1, 11)])
high_dimensional_df["group"] = high_dimensional_groups

# Calculate means for every group
means = high_dimensional_df.groupby('group').mean()

# Create spider chart
fig = go.Figure()

for group in means.index:
    fig.add_trace(go.Scatterpolar(
        r=means.loc[group].values,
        theta=means.columns,
        fill='toself',
        name=group
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True
)

fig.show()

#### Dimensionality reduction methods
Dimensionality reduction methods are a set of techniques used to reduce the number of variables in a high-dimensional dataset while preserving the most important information. These methods can be used to visualize high-dimensional data in a lower-dimensional space, making it easier to interpret and analyze.

One common dimensionality reduction method is principal component analysis (PCA), which is used to transform the original variables into a new set of variables that capture the most important information in the data. PCA works by finding the directions of maximum variance in the data, and projecting the data onto these directions. The resulting transformed data can be visualized in a lower-dimensional space, such as a 2D or 3D plot.

In [ ]:
from sklearn.decomposition import PCA

POPULATION_SIZE = 100
high_dimensional_population = np.random.rand(POPULATION_SIZE, 10)
high_dimensional_groups = np.random.choice(["A", "B", "C"], POPULATION_SIZE)

pca = PCA(n_components=2)
transformed_population = pca.fit_transform(high_dimensional_population)

fig = px.scatter(x=transformed_population[:, 0], y=transformed_population[:, 1], color=high_dimensional_groups) 
fig.update_layout(width=800, height=800)
fig.show()

## Genetic algorithm

We modify the GeneticAlgorithm:
- new field called `populations` - a list of populations from all epochs,
- new method called `_save_current_population`,
- adjusted `evaluate` method.

In [ ]:
from typing import TypeVar, List

from jmetal.config import store
from jmetal.core.algorithm import EvolutionaryAlgorithm
from jmetal.core.operator import Mutation, Crossover, Selection
from jmetal.core.problem import Problem
from jmetal.util.evaluator import Evaluator
from jmetal.util.generator import Generator
from jmetal.util.termination_criterion import TerminationCriterion

S = TypeVar('S')
R = TypeVar('R')

"""
.. module:: genetic_algorithm
   :platform: Unix, Windows
   :synopsis: Implementation of Genetic Algorithms.
.. moduleauthor:: Antonio J. Nebro <antonio@lcc.uma.es>, Antonio Benítez-Hidalgo <antonio.b@uma.es>
"""


class CustomGeneticAlgorithm(EvolutionaryAlgorithm[S, R]):

    def __init__(self,
                 problem: Problem,
                 population_size: int,
                 offspring_population_size: int,
                 mutation: Mutation,
                 crossover: Crossover,
                 selection: Selection,
                 termination_criterion: TerminationCriterion = store.default_termination_criteria,
                 population_generator: Generator = store.default_generator,
                 population_evaluator: Evaluator = store.default_evaluator):
        super(CustomGeneticAlgorithm, self).__init__(
            problem=problem,
            population_size=population_size,
            offspring_population_size=offspring_population_size)
        self.mutation_operator = mutation
        self.crossover_operator = crossover
        self.selection_operator = selection

        self.population_generator = population_generator
        self.population_evaluator = population_evaluator

        self.termination_criterion = termination_criterion
        self.observable.register(termination_criterion)

        self.mating_pool_size = \
            self.offspring_population_size * \
            self.crossover_operator.get_number_of_parents() // self.crossover_operator.get_number_of_children()

        if self.mating_pool_size < self.crossover_operator.get_number_of_children():
            self.mating_pool_size = self.crossover_operator.get_number_of_children()
        
        # Custom field:
        self.populations = []

    def create_initial_solutions(self) -> List[S]:
        return [self.population_generator.new(self.problem)
                for _ in range(self.population_size)]

    def evaluate(self, population: List[S]):
        ret = self.population_evaluator.evaluate(population, self.problem)
        return ret
    
    # Custom method:
    def _save_current_population(self, population: List[S]) -> None:
      self.populations.append([individual.variables for individual in population])

    def stopping_condition_is_met(self) -> bool:
        return self.termination_criterion.is_met

    def selection(self, population: List[S]):
        mating_population = []

        for i in range(self.mating_pool_size):
            solution = self.selection_operator.execute(population)
            mating_population.append(solution)

        return mating_population

    def reproduction(self, mating_population: List[S]) -> List[S]:
        number_of_parents_to_combine = self.crossover_operator.get_number_of_parents()

        if len(mating_population) % number_of_parents_to_combine != 0:
            raise Exception('Wrong number of parents')

        offspring_population = []
        for i in range(0, self.offspring_population_size, number_of_parents_to_combine):
            parents = []
            for j in range(number_of_parents_to_combine):
                parents.append(mating_population[i + j])

            offspring = self.crossover_operator.execute(parents)

            for solution in offspring:
                self.mutation_operator.execute(solution)
                offspring_population.append(solution)
                if len(offspring_population) >= self.offspring_population_size:
                    break

        return offspring_population

    def replacement(self, population: List[S], offspring_population: List[S]) -> List[S]:
        population.extend(offspring_population)

        population.sort(key=lambda s: s.objectives[0])

        new_population = population[:self.population_size]
        # Custom code:
        self._save_current_population(new_population)

        return new_population

    def get_result(self) -> R:
        return self.solutions[0]

    def get_name(self) -> str:
        return 'Genetic algorithm 1'

### Rastrigin function

The Rastrigin function is a commonly used test function in optimization and machine learning. It is a non-convex function that is often used to test the performance of optimization algorithms, as it has many local minima and is difficult to optimize.

The Rastrigin function is defined as:

$$ f(x) = 10d + \sum_{i=1}^d[x_i^2 - 10\cos{(2 \pi x_i)}] $$

The function is usually evaluated on the hypercube $ x_i \in [-5.12, 5.12]$, for all $i = 1, …, d$.

Global Minimum:

$f(x^*) = 0$, at $x^* = (0, … 0)$.


![rastrigin](https://www.sfu.ca/~ssurjano/rastr.png)

Rastrigin for $d=2$:

In [ ]:
from jmetal.problem.singleobjective.unconstrained import Rastrigin

d = 2
problem = Rastrigin(d)

In [ ]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.operator import BinaryTournamentSelection, PolynomialMutation, SBXCrossover
from jmetal.util.termination_criterion import StoppingByEvaluations

# Let us run the algorithm with artificial parameters:
POPULATION_SIZE = 4
OFFSPRING_POPULATION_SIZE = 4
MUTATION = PolynomialMutation(1.0 / problem.number_of_variables(), 20.0)
CROSSOVER = SBXCrossover(0.9, 20.0)
SELECTION = BinaryTournamentSelection()
MAX_EVALUATIONS = 10

algorithm = CustomGeneticAlgorithm(
  problem=problem,
  population_size=POPULATION_SIZE,
  offspring_population_size=OFFSPRING_POPULATION_SIZE,
  mutation=MUTATION,
  crossover=CROSSOVER,
  selection=SELECTION,
  termination_criterion=StoppingByEvaluations(max_evaluations=MAX_EVALUATIONS))


algorithm.run()

result = algorithm.get_result()

print('Algorithm: {}'.format(algorithm.get_name()))
print('Problem: {}'.format(problem.name()))
print('Solution: {}'.format(result.variables))
print('Fitness: {}'.format(result.objectives[0]))
print('Computing time: {}'.format(algorithm.total_computing_time))

Let's plot the first population:

In [ ]:
populations = [np.array(population) for population in algorithm.populations] 
first_population = populations[0]
plt.scatter(x=first_population[:, 0], y=first_population[:, 1])
plt.xlim(problem.lower_bound[0], problem.upper_bound[0])
plt.ylim(problem.lower_bound[1], problem.upper_bound[1])
plt.xlabel("x")
plt.ylabel("y")
plt.title("First population")
plt.show()

#### Heat map, contour plot

In [ ]:
def rastrigin(x, y):
    return 20 + x**2 - 10 * np.cos(2 * np.pi * x) + y**2 - 10 * np.cos(2 * np.pi * y)

x = np.linspace(problem.lower_bound[0], problem.upper_bound[0], 100)
y = np.linspace(problem.lower_bound[1], problem.upper_bound[1], 100)
X, Y = np.meshgrid(x, y)
Z = rastrigin(X, Y)

plt.contourf(X, Y, Z, 20, cmap="viridis")
plt.colorbar(label="Rastrigin function value")

plt.scatter(x=first_population[:, 0], y=first_population[:, 1], color="red")

plt.xlabel("x")
plt.ylabel("y")
plt.title("Contour plot of the Rastrigin function for first population")
plt.grid(True)
plt.axis("equal")
plt.show()

Exercise 1.
1. Visualize all `populations`, use selected chart and explain your choice.
1. Create a GIF that shows how the population is changing (use `matplotlib.animation` or `PIL.Image` or `imageio`) for contour plot.

Q: How to check if populations from different epochs are from the same distribution?

In [ ]:
from matplotlib.animation import FuncAnimation


# Create figure and axis
fig, ax = plt.subplots()
ax.contourf(X, Y, Z, 20, cmap="viridis")
scatter = ax.scatter(x=first_population[:, 0], y=first_population[:, 1], color="red")
ax.set_title(f"Population {0}")
scatters = [scatter]

# Update function for animation
def update(num):
    ax.clear()
    ax.set_title(f"Population {num}")
    ax.contourf(X, Y, Z, 20, cmap="viridis")
    scatter = ax.scatter(populations[num][:, 0], populations[num][:, 1], color="red")
    return scatters + [scatter]

    

# Create animation
ani = FuncAnimation(fig, update, frames=len(populations), blit=True)

# Save as GIF
ani.save('animation.gif', writer='imagemagick')

Exercise 2.

Investigate how number of evaluations and number of individuals affect solution. For every pair (`MAX_EVALUATIONS`, `POPULATION_SIZE`) measure time of execution and get the best fitness value. What are the best parameters regarding time and quality of solution? Create a GIF for this pair of parameters.

Try to use: 

`MAX_EVALUATIONS = [100, 200, 500, 1000, 2000, 5000]`

`POPULATION_SIZE = [10, 20, 50, 100]`

For `POPULATION_SIZE=100` plot best fitness (and time of execution) values by `MAX_EVALUATIONS`.

Q1: "We aim to achieve the best solution in the shortest possible time." How would you describe this optimization problem? <br>
Q2: What's the difference between `POPULATION_SIZE` and `OFFSPRING_POPULATION_SIZE`? Try to adjust (increase) `OFFSPRING_POPULATION_SIZE`. <br>
Q3: How can we improve the way we look for parameters?

Exercise 3. 

Execute the Genetic Algorithm on a 10-dimensional Rastrigin function. Generate a scatter plot for the first two dimensions of the last population (use `POPULATION_SIZE=100` and `MAX_EVALUATIONS=1000`). Then, apply Principal Component Analysis (PCA) to the last population and create a scatter plot of the transformed data. Compare the two scatter plots to analyze the differences.

Exercise 4.

Explore the impact of mutation on the solution. Increase/decrease mutation probability/rate. Create GIFs for smallest and largest mutation probability. What are the differences between them? Run algorithm for one of suggested $p_m$ values in the [article](https://www.sciencedirect.com/science/article/pii/089571779500035Z) (end of page 3). Compare results.

Exercise 5.

Quantitatively evaluate the diversity of populations from ex. 3. The goal is to develop a formula or metric that can be used to measure the diversity of the population, and to use this metric to assess the quality of the optimization process. Generate a simple plot showing the actual measure depending on evaluation number. Run algorithm multiple times and use box and whiskers to show the dispersion.


Extra exercise:
Consider measuring the diversity for the combinatorial and discrete problem.

Exercise 6.

Multi-start is a technique used in optimization to improve the chances of finding the global minimum of a function. The idea behind multi-start is to run a local optimization algorithm multiple times, each time starting from a different initial point in the search space. By doing this, we increase the chances of finding the global minimum, as each run of the local optimization algorithm may converge to a different local minimum.

The multi-start method works as follows:

1. Select a random starting point (e.g. with uniform distribution over domain).
2. For selected starting point, run the local optimization algorithm until convergence.
3. Record the best solution found across all runs of the local optimization algorithm.
4. Repeat 1-3 `N` times.

Implement multi start method. Use [L-BGS-B](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html) as a local method. Measure the number of fitness function evaluations, compare results with genetic algorithm.

